This notebook 

In [1]:
#import packages and initialize settings
import os
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii
from reproject import reproject_interp

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']="white"

import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
from SNeGenerationModels import runModels
from FindNearestMC import nearestMCMethod

In [2]:
# Get SNeCO information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")

database[0:1]

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits


In [3]:
galaxies = np.array(database["galaxy"], dtype='object')   
SNe, SNra, SNdec  = np.array(database["SN_name"], dtype='object'), np.array(database["SN_ra"], dtype='object'), np.array(database["SN_dec"], dtype='object')
telOrient = np.array(database["telOrient"], dtype='object')
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))] 

#fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_1kpc.fits"
fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_gauss7p5.fits"
hdulist = pyfits.open(fileName)
map = hdulist[0].data
#wcs = WCS(hdulist[0].header, naxis=2)



In [4]:
# generate list of galaxies in IRAC sample and Phangs Alma coverage maps
# "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "_12m+7m+tp_co21_150pc_coverage2d.fits"

IRFileList, PACovFileList = [],[]

for i in range(len(galaxies)):
    PACovFileList.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxies[i] + "/" + galaxies[i] + "_" + telOrient[i] + "_co21_150pc_coverage2d.fits")
    if galaxies[i] == "circinus":
        galaxies[i] = "eso097-013"
    fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/" + galaxies[i] + "_irac1_gauss7p5.fits"
    if os.path.isfile(fileCheck) == False:
        fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/" + galaxies[i] + "_w1_gauss7p5.fits"
        if os.path.isfile(fileCheck) == False:
            fileCheck = "No IRAC or WISE File for " + galaxies[i] + "."        
    else: pass
    IRFileList.append(fileCheck)

    

In [5]:
# for i in range(len(fileList)):
#     print(fileList[i])

# database.add_column(PACovFileList, name ="PACovFile")
# database.add_column(IRFileList, name = "IFRFile")

database[0:1]

# ascii.write(database, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits


Missing IRAC files for Circinus, NGC2997, NGC4945, NGC5128, NGC5530, NGC6744
Wise files available in ZOMGS directory
Missing 150pc res and coverage files for NGC 1068, 1672, & 4579
for now will leave out and maybe revisit with another resolution later?

What resolution is this map: 7p5 arcsec
W1 wise is 3.4 microns.
"IRAC is a four-channel camera that provides simultaneous 5.2 x 5.2 arcmin images at 3.6, 4.5, 5.8, and 8 microns (the black instrument in the lower right part of the above image; see the optical housing model and the conceptual layout diagram. The pixel size is 1.22 arcsec in all bands. "

In [6]:
# remove duplicate galaxies, grouping SNe together in their own lists.

gal_group = database.group_by("galaxy")

SNr, SNd, SNa = [],[],[]
gala, ifrs, covs, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea = [],[],[],[],[],[],[],[],[],[],[],[],[]

for i in range(len(database)):
    gal = database["galaxy"][i]
    ifr = database["IFRFile"][i]
    cov = database["PACovFile"][i]
    img = database["ImageFile150"][i]
    ers = database["ErrFile150"][i]
    aco = database["AlphaCOFile"][i]
    ccs = centerCoords[i]
    pas = database["orient_posang"][i]
    inc = database["orient_incl"][i]
    gds = database["dist"][i]
    SNr.append(SNra[i])
    SNd.append(SNdec[i])
    SNa.append(SNe[i])
    
    if (i+1) in gal_group.groups.indices:
        #print(gal)
        if gal != "ngc1068" and gal != "ngc1672" and gal != "ngc4579":
            #print(gal)
            gala.append(gal)
            ifrs.append(ifr)
            covs.append(cov)
            imgs.append(img)
            errs.append(ers)
            alco.append(aco)
            cocs.append(ccs)
            pans.append(pas)
            incl.append(inc)
            gdis.append(gds)
            SNrs.append(SNr)
            SNdc.append(SNd)
            SNea.append(SNa) 
            SNr, SNd, SNa = [],[],[]
        else: pass
    else: pass

In [9]:
# Next apply clipping to match field of coverage
# use reproject_interp and project IRAC onto the PHANGS one and keep the overlap coverage as the locations that 
# the SNe can occur in 

# gala, ifrs, covs, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea



with open("../Data/NearestMC_Stellar.txt", "w") as textfile:
    print("#Galaxy, dist to neares 10^5.5 MC, dist to nearest 10^6.5 MC", file = textfile)
    #for i in range(1):

    for i in range(len(gala)):
        if incl[i] == 90.0:
            incl[i] = 80.0
        print(gala[i])
        
        covFile = covs[i]
        ifrFile = ifrs[i]
        hdu_cov = pyfits.open(covFile)
        hdu_ifr = pyfits.open(ifrFile)
        covMap  = hdu_cov[0].data
        ifrMap, footprint = reproject_interp(hdu_ifr, hdu_cov[0].header)

        #wcs     = WCS(hdu_ifr[0].header, naxis=2)
        wcs     = WCS(hdu_cov[0].header, naxis=2)

        naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
        grid    = np.indices((naxis[1],naxis[0]))
        ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 
        
        f_cov = covMap.flatten()
        f_ifr = ifrMap.flatten()  
        f_ra  = ra.flatten()
        f_dec = dec.flatten()
        
        keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
        ifr  = f_ifr[keep]
        ra   = f_ra[keep]
        dec  = f_dec[keep]

        starRandra, starRanddec, starRanddx, starRanddy = runModels(gala[i], imgs[i], cocs[i], pans[i], incl[i], gdis[i], modelType = 3, starLight = ifr, starRa = ra, starDec = dec)

        
        nearest55, nearest65 = nearestMCMethod(gala[i], imgs[i], errs[i], alco[i], cocs[i], pans[i], incl[i], gdis[i], starRandra, starRanddec)
            
#Clean this up        #if np.isnan(nearest55):
        print(gala[i], nearest55, nearest65, file = textfile)




    

circinus
at Mass cutoff A for circinus
Nearest 55 [0.00082147 0.         0.01193642 0.         0.         0.
 0.         0.         0.00081927 0.         0.         0.00082051
 0.         0.         0.00532    0.00082137 0.         0.
 0.         0.         0.         0.         0.         0.0313679
 0.         0.         0.         0.00503126 0.         0.00354221
 0.         0.02332327 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00531672 0.
 0.         0.0114687  0.         0.00661383 0.         0.
 0.         0.         0.00470764 0.         0.         0.00882613
 0.         0.00293272 0.         0.         0.00088673 0.
 0.         0.00294014 0.         0.         0.00163828 0.00132279
 0.         0.00293512 0.         0.00264486 0.         0.
 0.00440145 0.         0.         0.         0.00132229 0.00088683
 0.01487088 0.0017

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


at Mass cutoff A for ngc0253
Nearest 55 [0.         0.         0.         0.         0.         0.
 0.         0.00087487 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01002046 0.         0.00138114 0.         0.
 0.         0.         0.00087499 0.         0.         0.
 0.         0.         0.         0.         0.0015237  0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.03146725 0.         0.
 0.         0.         0.00708681 0.00174568 0.         0.0027627
 0.         0.         0.         0.         0.         0.00087337
 0.00206593 0.         0.         0.         0.00173667 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.00262068
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00277972 0.
 0.         0.         0.         0.         0.00275

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
at Mass cutoff A for

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
at Mass cutoff A for

...

at Mass cutoff A for ngc4457
Nearest 55 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.31975297e-03
 1.78554274e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.19619585e-04 2.50470677e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.13333290e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.86972574e-04 2.45221422e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.61538341e-04
 0.00000000e+00 0.00000000e+00 1.75753800e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.77733521e-03 0.00000000e+00 1.40296563e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.34904990e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.87574447e-03 0.00000000e+00 8.34904537e-05 3.48717812e-04
 0.00000000e+00 6.97435679e-04 1.39191090e-03 0.00000000e+00
 0.00000000e+00 2.36275707e-03 5.92448811e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.66981096e-04 0.00000000e+00
 0.00000000e+00 1.86971300e-04 0.00000000e+00

at Mass cutoff A for ngc4536
Nearest 55 [0.         0.         0.00028651 0.         0.         0.00017351
 0.00354416 0.         0.00311173 0.00255896 0.00495811 0.00054676
 0.00068925 0.00034702 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00027434 0.
 0.00170775 0.00012553 0.00012553 0.00390757 0.         0.0003766
 0.         0.         0.         0.         0.0033106  0.00329666
 0.         0.         0.00364871 0.00087242 0.00498166 0.0021947
 0.00569105 0.00098438 0.         0.00063234 0.00112058 0.00168952
 0.         0.         0.00908727 0.         0.         0.00037659
 0.0009039  0.0003013  0.         0.00025106 0.00639768 0.
 0.         0.         0.00683192 0.         0.00098438 0.
 0.00244502 0.         0.00100614 0.0012497  0.         0.
 0.         0.00145573 0.         0.         0.         0.
 0.         0.         0.         0.         0.00012553 0.
 0.         0.       

at Mass cutoff A for ngc5128
Nearest 55 [0.         0.         0.         0.         0.00140277 0.00417756
 0.01196232 0.01099637 0.         0.         0.         0.
 0.00279751 0.         0.         0.         0.         0.00063094
 0.         0.02904984 0.         0.         0.         0.
 0.01171916 0.00124835 0.00292687 0.         0.         0.
 0.0011036  0.         0.0011039  0.00063123 0.         0.
 0.00438688 0.         0.00417744 0.         0.04217802 0.01222475
 0.         0.         0.         0.00428334 0.         0.
 0.00241551 0.         0.         0.         0.         0.00200385
 0.         0.         0.00400836 0.00063091 0.         0.
 0.         0.         0.0083603  0.00772453 0.00529074 0.01142922
 0.00126189 0.         0.         0.01325353 0.00441323 0.
 0.         0.         0.00161034 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00912147 0.         0.         0.         0.
 0.         0.         0.         0

at Mass cutoff A for ngc5530
Nearest 55 [0.01057103 0.         0.00096923 0.00293951 0.00039197 0.00024692
 0.         0.00047773 0.         0.00107319 0.         0.00054064
 0.         0.00024593 0.         0.00476542 0.00058182 0.00047774
 0.         0.00159832 0.00030029 0.         0.         0.
 0.00132504 0.00012345 0.         0.         0.         0.00024692
 0.00125158 0.00024591 0.         0.         0.         0.00019387
 0.         0.         0.         0.00310456 0.         0.00049161
 0.00038783 0.         0.00486509 0.         0.         0.00019385
 0.00101503 0.0002459  0.0012373  0.00049036 0.         0.00047965
 0.00060182 0.         0.         0.00232614 0.00068262 0.00155213
 0.         0.00012294 0.0010697  0.         0.00453952 0.00036881
 0.00086042 0.00117585 0.         0.         0.00030093 0.
 0.         0.         0.         0.00024829 0.         0.
 0.         0.         0.00012346 0.         0.         0.
 0.         0.00103174 0.00095322 0.00114535 0.0016044

at Mass cutoff A for ngc7793
Nearest 55 [0.0023081  0.00355218 0.00059996 0.0035186  0.0011565  0.00924416
 0.00125345 0.0030647  0.00083406 0.02224868 0.03062506 0.00166808
 0.00930627 0.00299349 0.00287863 0.002879   0.00649959 0.0133394
 0.00192519 0.0074772  0.         0.00115649 0.00345923 0.
 0.00115644 0.00492862 0.00160588 0.00299405 0.03167906 0.00768819
 0.01921312 0.00332088 0.00351884 0.0021864  0.02367605 0.00250755
 0.00832321 0.00321234 0.00175787 0.02790439 0.0041619  0.00287775
 0.         0.01292129 0.00060002 0.00747811 0.01792757 0.02149571
 0.00060011 0.00612994 0.00175776 0.01418978 0.00087964 0.00332133
 0.01500215 0.03179748 0.01865381 0.00240319 0.01900249 0.00380654
 0.01495094 0.00416247 0.         0.0016061  0.02848666 0.02838659
 0.00164266 0.01079712 0.         0.00119989 0.00175788 0.02129866
 0.         0.0012538  0.         0.00834069 0.00978982 0.00575628
 0.03066642 0.         0.00160581 0.00416278 0.00230866 0.
 0.01476907 0.0008341  0.00578325 0.001

In [8]:
print(nearestMCs)

NameError: name 'nearestMCs' is not defined

In [ ]:
#from FindNearestMC import findNearest

# def findNearest(x1,x2,y1,y2):
#     return(mindist) 

# if this uses too much memory, play minesweeper, 0 for gas and grow
# scipy scikit-image

# or monte carlo and only do numpy random choice to pick weigheted probability choice from the probability arrays 
# and then do like 10,000 draws from that array before feeding into the pair_dist_calc


# can imshow the matricies 
# print the shapes of the tile columns, etc
